# OPEN STREET MAP PROJECT (CITY OF HOUSTON)


In this project I used sql and converted the query results into pandas data frame for the purpose of clear tabular 
visualization in the notebook.

In [2]:
import sqlite3
import pandas as pd
import numpy as np
import csv
import re
from itertools import islice
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

# OpenStreetMap Data Project

Data: The extracted dat is the open street map data for the Greater Houston Area. Houston is my home city and I was 
curious to investigate the data. The data link is https://mapzen.com/data/metro-extracts/metro/houston_texas/



File size =468.4 MB (unzipped)

In [3]:
filename="/Users/eyobe/Desktop/UDML/analysis/openstreet/nodes_tags.csv"

# Problems encountered in the data


 After extracting the nodes, nodes_tags, ways,ways_tags,and ways_nodes from xml to csv format I encountered the following problems.
 
1. Data type inconsistency: 
The field values of the nodes.csv file were not in the correct format  of my        sql     schema to load into sql table. I fixed the problem  by converting the field values into     their correct types and also I found that the primary keys in the all 4 of the files       were not unique, that led me to remove the primary key constraint from my tables'   schema. The code to convert the field values into the correct types is using node_and_way_reshaping.py file and the schema of my sql tables are in sql_schema.py file.
 
2. Data values in other languages : 
The nodes_tags csv file has enteries written in different languages other than English, I cleaned out that befor I load to the sql table. There were about 50 field values written in other languages here I printed out 5  of them as example from the csv file and laso in DataFrame format. The main code I used to clean is found in auditing_city_and_street.py file.
 

In [4]:
with open (filename,"r") as f:
    reader=csv.DictReader(f)
    header=reader.fieldnames
    for line in islice(reader, 5,10):
        print line 
        print "\n"

{'value': '\xe1\x88\x82\xe1\x8b\x8d\xe1\x88\xb5\xe1\x89\xb0\xe1\x8a\x95', 'type': 'name', 'id': '27526178', 'key': 'am'}


{'value': '\xd9\x87\xd9\x8a\xd9\x88\xd8\xb3\xd8\xaa\xd9\x86\xd8\x8c \xd8\xaa\xd9\x83\xd8\xb3\xd8\xa7\xd8\xb3', 'type': 'name', 'id': '27526178', 'key': 'ar'}


{'value': 'Hyuston', 'type': 'name', 'id': '27526178', 'key': 'az'}


{'value': "\xd0\xa5'\xd1\x8e\xd1\x81\xd1\x82\xd0\xb0\xd0\xbd", 'type': 'name', 'id': '27526178', 'key': 'be'}


{'value': '\xd0\xa5\xd1\x8e\xd1\x81\xd1\x82\xd1\x8a\xd0\xbd', 'type': 'name', 'id': '27526178', 'key': 'bg'}




In [5]:
nodes_tags_df= pd.read_csv(filename)
nodes_tags_df[5:10]

,id,key,value,type
5,27526178,am,ሂውስተን,name
6,27526178,ar,هيوستن، تكساس,name
7,27526178,az,Hyuston,name
8,27526178,be,Х'юстан,name
9,27526178,bg,Хюстън,name


3.Over abberevated street names: 
Some street names are over abbrevated for like, St instead of street, Ave, instead of Avenue, etc.. and the  writing was not uniform in some of the street names. I cleaned these irregularities and modified the street names with a complete name except those without specified street types(there are streets with only name but no type i.e not known whether they are street, avenue...etc). The code I used to clean street names and city names before I write to the sql table is given in auditing_city_and_street.py file.

4.Inconsistent zip codes and non unique zipcode keys: 
The Zip code keys in the
data under nodes_tags and way_tags are; 'postcode','postal_code','Zipcode'  
'zip_right','zip_left_1','zip_left_2','zip_right_1','zip_left'. 
I wrote a function to find  inconsistent zip codes from node_tags and way_tags. The following were invalid zipcods I found.['Weslayan Street', '88581', 'TX 77494', 'TX 77009', 'TX 77086', '75057', '7-', 'TX 77005']. Though the zipcodes 77494,77009,77086 are valid Houston zip codes, they all preceded by TX and categorized as invalid. The full code I used to I used to extract valid and Invalid zipcodes is in zipcode_auditing.py file and the function  is given below.


In [47]:

"""THIS FUNCTION CLEANS AND EXTRACTS PROBLEMATIC ZIP CODES """
def clean_zip_code(z):
    validcode=[]
    invalidcode=[]
    for value in z:
         #  US zipcodes  are usually five digit and sometimes may may be written as xxxxx-yyyy where 
            #xs and ys are integers.
        if (len(value)<5) or (not value.split("-")[0].isdigit()):  
            invalidcode.append(value)
        #print value[0:2]
        elif value.split("-")[0].isdigit() and int(value[0:2])!=77: #Houston, Texas zip code
             invalidcode.append(value)
        else:
             validcode.append(value)
    return invalidcode,validcode

5.Problems in the city names:  I found the same city name written in different ways for example Houston was written in the following different formats; "Houston","Houston, Texas","Houston, TX","HOUSTON", the same city but some how different names example ("The Woodlands" and, "Woodlands"), with also spelling error "Houson". I found this type of non uniformity and also spelling errors in all other cities for example "Dickinson, Tx","Dickinson","Dickenson" for Dickinson city. I try to fix the non uniformity by writing the city into similar format of the form City, State, Country format before I load to sql table. The main code I used is auditing_city_and_street.py file.

In [48]:
#def clean_city(city):
    #city=city.split(",")
    #city_with_state_country=city[0].capitalize()+','+'Texas'+','+'USA'
    #return city_with_state

# Analaysis of the data using SQL
The schema of my sql tables and the codes to import from csv to sql are given in the sql_schema.py file. 
Here I used sqlite3 API to perform queries on this jupyter note book and I wrote  a simple function that display the results of the sql queries in to pandas data frame for only purpose of clear tabular presentation. The code to read sql queries into pandas data frame is given in  sql_to_dataframe.py file and also given here in the notebook below.

In [6]:
#Function to  convert my sql queries to pandas dDataFrame.
def sql_to_df(sql_query):

    # Use pandas to pass sql query using connection form SQLite3
    df = pd.read_sql(sql_query, con)
    #df.index= False
    # Show the resulting DataFrame
    return df

# CREATING SQL CONNECTION

In [51]:
con = sqlite3.connect('openstreet.db')


# Number of Nodes 


In [50]:
query='''SELECT COUNT(*)  as Total_number_of_nodes FROM new_nodes;
'''
sql_to_df(query)

,Total_number_of_nodes
0,1956751


# Number of Ways

In [54]:
query='''SELECT COUNT(*) as Total_number_of_ways FROM new_ways;
'''

sql_to_df(query)

,Total_number_of_ways
0,10220


# Total Number of contributions

In [56]:
query='''SELECT COUNT(*) AS Total_number_of_contributions FROM (SELECT uid FROM new_nodes UNION ALL SELECT uid FROM new_ways);
'''
sql_to_df(query)

,Total_number_of_contributions
0,1966971


In [33]:
#qq='''SELECT * FROM new_nodes LIMIT 5;'''

#sql_to_df(qq)

# Top 10 Contributers

In [57]:
query=''' SELECT DISTINCT uid, user, COUNT(*) as num
FROM (SELECT uid,user FROM new_nodes UNION ALL SELECT uid,user FROM new_ways) 
GROUP BY user
ORDER BY num DESC
LIMIT 10;'''

sql_to_df(query)


,uid,user,num
0,147510,woodpeck_fixbot,378618
1,1110270,afdreher,356686
2,3119079,cammace,165690
3,496606,scottyc,160336
4,9065,brianboru,107868
5,119881,claysmalley,87596
6,475877,RoadGeek_MD99,67567
7,243003,skquinn,60580
8,672878,TexasNHD,47051
9,2176227,Memoire,46787


# Number of unique contributers

In [43]:
query=''' SELECT DISTINCT uid, user, COUNT(*) as num
FROM (SELECT uid,user FROM new_nodes UNION ALL SELECT uid,user FROM new_ways) 
GROUP BY user
ORDER BY num DESC;'''
sql_to_df(query)
print(len(sql_to_df(query)))

1082


# Number of unique contributers contributed only once

In [58]:
query='''SELECT COUNT(*) as contributers_only_once
      FROM
    ( SELECT DISTINCT uid, user, COUNT(*) as num
    FROM (SELECT uid,user FROM new_nodes UNION ALL SELECT uid,user FROM new_ways)  
     GROUP BY user
     HAVING num=1) '''
sql_to_df(query)

,contributers_only_once
0,226


# Contributers statistics

First top 5 contributer contribution (%) 19.248784, 18.133770, 8.423612,    8.151417, 5.483965, 4.453345, 3.435079, 3.079862, 2.392054, 2.37863 and  Combined Contribution of ~ 75.18 %. The remaining ~ 24.82 % is contributed by 1072 contributors. This shows that a significant part of the map was contributed by the top 10 contributors combined.  

# Name of cities in greater Houston.

In [75]:
query='''SELECT DISTINCT key, value FROM cleaned_nodetags WHERE key="city" union 
SELECT DISTINCT key, value FROM cleaned_waytags WHERE key="city";'''
print "The cities in  greater Houston  area are :"
sql_to_df(query)


The cities in  greater Houston  area are :


,key,value
0,city,"Atascocita,Texas,USA"
1,city,"Baytown,Texas,USA"
2,city,"Bellaire,Texas,USA"
3,city,"Channelview,Texas,USA"
4,city,"Conroe,Texas,USA"
5,city,"Crosby,Texas,USA"
6,city,"Cypress,Texas,USA"
7,city,"Deer park,Texas,USA"
8,city,"Dickenson,Texas,USA"
9,city,"Dickinson,Texas,USA"


From the above result, we can see that some cities are repeated due to the non unique representation in the open street map data. These include;
1. Houson,Texas,USA and Houston,Texas,USA  
2. Woodlands,Texas,USA and The woodlands,Texas,USA 
3. West university place,Texas,USA and West university,Texas,USA
4. Dickinson,Texas,USA and Dickenson,Texas,USA
5. Sugar land,Texas,USA and Sugarland,Texas,USA


# Name of counties

In [15]:
query='''SELECT DISTINCT key, value FROM cleaned_nodetags WHERE key="county_name" union 
SELECT DISTINCT key, value FROM cleaned_waytags WHERE key="county";'''
print "The counties included in the data are :"
sql_to_df(query)



The counties included in the data are :


,key,value
0,county,"Harris, TX"
1,county,"Harris, TX:Liberty, TX"
2,county,"Harris, TX; Fort Bend, TX"
3,county_name,Brazoria
4,county_name,Camp
5,county_name,Chambers
6,county_name,Fort Bend
7,county_name,Galveston
8,county_name,Harris
9,county_name,Houston


The Above result also shows the countys in the open street map data were written in a non uniform way

# Top 10 ammenities 

In [65]:
query='''SELECT DISTINCT key, value,count(*) AS number FROM cleaned_nodetags WHERE 
key="amenity" GROUP BY value 
UNION ALL
SELECT DISTINCT key, value,count(*) AS number FROM cleaned_waytags WHERE 
key="amenity" GROUP BY value ORDER BY number DESC LIMIT 10;'''

sql_to_df(query)

,key,value,number
0,amenity,place_of_worship,1877
1,amenity,school,792
2,amenity,fountain,657
3,amenity,restaurant,607
4,amenity,fast_food,534
5,amenity,fire_station,265
6,amenity,fuel,226
7,amenity,pharmacy,157
8,amenity,bank,153
9,amenity,bench,126


In [18]:
#query='''SELECT DISTINCT id, key, value,count(*) AS number FROM cleaned_nodetags  
#WHERE key='feature_type' GROUP BY value ORDER BY number DESC LIMIT 100
#;'''
#print "The number of amenities in Harris County are :"
#sql_to_df(query)

# Top 5 Leisure Places

In [66]:
query=''' SELECT DISTINCT id,key, value,count(*) as number FROM cleaned_nodetags WHERE 
key="leisure" GROUP BY value 
UNION ALL
SELECT DISTINCT id,key, value,count(*) as number FROM cleaned_waytags WHERE 
key="leisure" GROUP BY value
ORDER BY number DESC LIMIT 5;'''
print "The 5 most common leisures :"
sql_to_df(query)

The 5 most common leisures :


,id,key,value,number
0,4251825278,leisure,park,186
1,4470575628,leisure,picnic_table,48
2,4494039339,leisure,slipway,27
3,4289031161,leisure,playground,26
4,4414027268,leisure,sports_centre,25


In [20]:
#query=''' SELECT DISTINCT id,key, value,count(*) as number FROM cleaned_nodetags WHERE 
#key="waterway" GROUP BY value ORDER BY number DESC LIMIT 10;'''
#print "The 10 most tourism  are :"
#sql_to_df(query)


# Population of Cities and Villages In Greater Houston area based on 2006 census 

In [68]:

query='''SELECT DISTINCT(id),key, value FROM cleaned_nodetags WHERE key ="population" GROUP BY id
UNION SELECT DISTINCT(id),key, value FROM cleaned_waytags WHERE key ="population" GROUP BY id ; '''
df=sql_to_df(query)

dff=(df['id']).unique()#.stack().to_frame().T
dff

array([  27526178,  151335482,  151336611,  151336814,  151339751,
        151352050,  151356757,  151364363,  151366400,  151370505,
        151378972,  151388438,  151413262,  151418385,  151432146,
        151449285,  151449547,  151453168,  151459173,  151469708,
        151487687,  151488459,  151492991,  151516758,  151520770,
        151524671,  151556974,  151559999,  151565543,  151633490,
        151658563,  151661012,  151679582,  151699317,  151725826,
        151742429,  151755512,  151807948,  151830242,  151830706,
        151841886,  151853310,  151853324,  151859424,  151872763,
        151882112,  151903369,  151912936,  151960564, 1302494067,
       2336724814, 3811458395])

In [73]:
query=''' SELECT * FROM cleaned_nodetags WHERE id IN 
(27526178,  151335482,  151336611,  151336814,  151339751,
        151352050,  151356757,  151364363,  151366400,  151370505,
        151378972,  151388438,  151413262,  151418385,  151432146,
        151449285,  151449547,  151453168,  151459173,  151469708,
        151487687,  151488459,  151492991,  151516758,  151520770,
        151524671,  151556974,  151559999,  151565543,  151633490,
        151658563,  151661012,  151679582,  151699317,  151725826,
        151742429,  151755512,  151807948,  151830242,  151830706,
        151841886,  151853310,  151853324,  151859424,  151872763,
        151882112,  151903369,  151912936,  151960564, 1302494067,
       2336724814, 3811458395) AND (key ='name' OR key='population') ;'''
df=sql_to_df(query)
#df.pivot(index='id', columns='key', values='value')
#print df
#dff=df.groupby(id)
df1=df[df.key=='population']
df2=df[df.key=='name']
df3=pd.merge(df1, df2, on='id')

df3.columns=['id','Kind', 'Population', 'Type','City Name','City','Type2']
df3=df3[df3.Type=='regular']
df3.Population=df3['Population'].astype(int)
df3.sort_values(['Population'],ascending=True).head()
#df3[['id','City','Population' ]]



,id,Kind,Population,Type,City Name,City,Type2
28,151449285,population,315,regular,name,Thompsons,regular
13,151364363,population,489,regular,name,Orchard,regular
61,151679582,population,517,regular,name,Stagecoach,regular
57,151658563,population,693,regular,name,Beasley,regular
69,151807948,population,727,regular,name,Hilshire Village,regular


# 10 Most Populous Cities or Villages in Greater Houston Area

In [75]:
query=''' SELECT * FROM cleaned_nodetags WHERE id IN 
(27526178,  151335482,  151336611,  151336814,  151339751,
        151352050,  151356757,  151364363,  151366400,  151370505,
        151378972,  151388438,  151413262,  151418385,  151432146,
        151449285,  151449547,  151453168,  151459173,  151469708,
        151487687,  151488459,  151492991,  151516758,  151520770,
        151524671,  151556974,  151559999,  151565543,  151633490,
        151658563,  151661012,  151679582,  151699317,  151725826,
        151742429,  151755512,  151807948,  151830242,  151830706,
        151841886,  151853310,  151853324,  151859424,  151872763,
        151882112,  151903369,  151912936,  151960564, 1302494067,
       2336724814, 3811458395) AND (key ='name' OR key='population') ;'''
df=sql_to_df(query)
df1=df[df.key=='population']
df2=df[df.key=='name']
df3=pd.merge(df1, df2, on='id')
df3.columns=['id','Kind', 'Population', 'Type','City Name','City','Type2']
df3=df3[df3.Type=='regular']
df3.Population=df3['Population'].astype(int)
df3[['id','City','Population' ]].sort_values(by='Population',ascending=0)[:10]



,id,City,Population
1,151335482,Pasadena,144793
90,1302494067,The Woodlands,93847
67,151755512,Pearland,91000
77,151853324,League City,83560
92,3811458395,Kingwood,81692
22,151413262,Sugar Land,79943
31,151453168,Baytown,75418
20,151388438,Spring,54298
70,151830242,Friendswood,37587
53,151565543,La Porte,33886


In [82]:

#sns.barplot(x="City", y="Population",  data=df3);

$\textbf{The Most Populous city in the greater Houston  area is $\textit{Pasadena}$ and the second populous is $\textit{The Woodlands}$}$

# 5 Least Populated Places in the greater Houston area according to 2006 census

In [24]:
df3[['id','City','Population' ]].sort_values(by='Population',ascending=0).tail()

,id,City,Population
69,151807948,Hilshire Village,727
57,151658563,Beasley,693
61,151679582,Stagecoach,517
13,151364363,Orchard,489
28,151449285,Thompsons,315


# 5 Most Popular religions

In [76]:
query='''SELECT DISTINCT *, count(value) as num
    FROM cleaned_nodetags 
    WHERE  key='religion' GROUP BY value
    UNION ALL
    SELECT DISTINCT *, count(value) as num
    FROM cleaned_waytags 
    WHERE  key='religion' GROUP BY value
    ORDER BY num DESC LIMIT 5;'''
sql_to_df(query)

,id,key,value,type,num
0,4442593176,religion,christian,regular,1820
1,356866689,religion,buddhist,regular,14
2,3857010081,religion,jewish,regular,10
3,4214577163,religion,muslim,regular,9
4,356866022,religion,unitarian_universalist,regular,4


$\textbf{Christian is by far the most popular religion}$ 

# 10 Most Popular Chrstian Religions by denomination

In [79]:
query='''SELECT DISTINCT *, count(value) as num
    FROM cleaned_nodetags 
    WHERE  key='denomination' GROUP BY value 
    UNION ALL
    SELECT DISTINCT *, count(value) as num
    FROM cleaned_waytags 
    WHERE  key='denomination' GROUP BY value
    ORDER BY num DESC LIMIT 5;'''
sql_to_df(query)
sql_to_df(query)

,id,key,value,type,num
0,4026885902,denomination,baptist,regular,820
1,3380048293,denomination,methodist,regular,126
2,4306416599,denomination,pentecostal,regular,91
3,4309418044,denomination,lutheran,regular,69
4,356866648,denomination,catholic,regular,61


$\textit{Baptist Christian is the most popular from Christian religion}$

# 10 Popular Cuisines 

In [80]:
query='''SELECT DISTINCT *, count(value) as num
    FROM cleaned_nodetags 
    WHERE  key='cuisine'
 GROUP BY value 
 UNION ALL
 SELECT DISTINCT *, count(value) as num
    FROM cleaned_waytags 
    WHERE  key='cuisine'
 GROUP BY value 
 ORDER BY num DESC  LIMIT 10;'''
sql_to_df(query)

,id,key,value,type,num
0,4469127878,cuisine,burger,regular,168
1,4509228265,cuisine,sandwich,regular,108
2,4504777180,cuisine,mexican,regular,100
3,4509228262,cuisine,pizza,regular,65
4,4451815138,cuisine,chicken,regular,53
5,4446394698,cuisine,coffee_shop,regular,40
6,4468775199,cuisine,american,regular,35
7,4468726393,cuisine,italian,regular,32
8,4439886289,cuisine,chinese,regular,29
9,4469127881,cuisine,ice_cream,regular,15


In [28]:
# 5 Most Tourism places

In [29]:
#query='''SELECT DISTINCT *, count(value) as num
#    FROM cleaned_nodetags 
#    WHERE  key='tourism' GROUP BY value ORDER BY num DESC LIMIT 5;'''
#sql_to_df(query)



# 5 Common land uses

In [81]:
query='''SELECT DISTINCT *, count(value) as num
    FROM cleaned_nodetags 
    WHERE  key='landuse' GROUP BY value 
    UNION ALL
    SELECT DISTINCT *, count(value) as num
    FROM cleaned_waytags 
    WHERE  key='landuse' GROUP BY value
    ORDER BY num DESC LIMIT 5;'''
sql_to_df(query)



,id,key,value,type,num
0,663736294,landuse,industrial,regular,80
1,356786985,landuse,reservoir,regular,38
2,1818344910,landuse,quarry,regular,26
3,2772809623,landuse,retail,regular,5
4,1143263540,landuse,recreation_ground,regular,3


# Common Institutions by City in greater Houston area

1. Name and number of pharmacies  by city in greater Houston area

In [106]:
q='''SELECT * FROM  cleaned_nodetags WHERE id IN (SELECT id FROM cleaned_nodetags WHERE value ='pharmacy'); '''
dfp1=sql_to_df(q)
q1='''SELECT * FROM  cleaned_nodetags WHERE id IN (SELECT id FROM cleaned_nodetags WHERE value = 'pharmacy') 
AND key='city' ; '''
dfp2=sql_to_df(q1)
#dfp2
q2='''SELECT * FROM  cleaned_nodetags WHERE  id IN (SELECT id FROM cleaned_nodetags WHERE value = 'pharmacy') 
AND key='name'; '''
dfp3=sql_to_df(q2)
dfp3
df4=pd.merge(dfp1,dfp2, on='id')
#df5=pd.merge(df4,df3,on='id')
df4.columns=['id','Key','Count','Type','Place',"City",'Type2']
#['value_y'].value_counts()
df5=pd.merge(df4,dfp3,on='id')

df6=df5[['Count','City','value']]
#df6.columns=['City','Name','Count']
df7=df6.groupby(['City','value']).count()
#df7=df7.rename(columns={'City':'City', 'value':'School Name', 'Amenity Type':'Count'})
df7


Count
City                    value                         
Bellaire,Texas,USA      Randalls Pharmacy            8
Cypress,Texas,USA       H-E-B Pharmacy              22
                        Randalls Pharmacy            8
Deer park,Texas,USA     H-E-B Pharmacy               8
Friendswood,Texas,USA   H-E-B Pharmacy               7
Houston,Texas,USA       CVS                          7
                        CVS Pharmacy                13
                        H-E-B Pharmacy             108
                        Randalls Pharmacy          104
                        Walgreens                   28
Humble,Texas,USA        H-E-B Pharmacy               7
Katy,Texas,USA          H-E-B Pharmacy              21
                        Randalls Pharmacy           24
Kingwood,Texas,USA      H-E-B Pharmacy               7
                        Kelsey-Seybold Pharmacy      9
                        Kroger Pharmacy              8
                        Randalls Pharmacy           16
                        Walgreens                    7
League city,Texas,USA   H-E-B Pharmacy               7
                        Randalls Pharmacy            8
                        Walgreens                    7
Missouri city,Texas,USA CVS                          7
                        H-E-B Pharmacy              14
                        Walgreens                   14
Pasadena,Texas,USA      H-E-B Pharmacy               7
Pearland,Texas,USA      H-E-B Pharmacy              15
                        Randalls Pharmacy            8
Richmond,Texas,USA      Kroger Pharmacy              9
                        Randalls Pharmacy            8
Seabrook,Texas,USA      CVS                          7
                        Walgreens                    7
Spring,Texas,USA        H-E-B Pharmacy              36
                        Randalls Pharmacy            8
Stafford,Texas,USA      Randalls Pharmacy            8
Sugar land,Texas,USA    H-E-B Pharmacy              14
                        Randalls Pharmacy            8
The woodlands,Texas,USA H-E-B Pharmacy               7
                        Randalls Pharmacy            8
Tomball,Texas,USA       H-E-B Pharmacy               9
                        Walgreens                    5

In [116]:
q='''SELECT * FROM  cleaned_nodetags WHERE id IN (SELECT id FROM cleaned_nodetags WHERE value LIKE '%pharmacy%'); '''
sql_to_df(q).head()

,id,key,value,type
0,461853706,name,Walgreens,regular
1,461853706,amenity,pharmacy,regular
2,472496331,name,Walgreens,regular
3,472496331,amenity,pharmacy,regular
4,504536337,amenity,pharmacy,regular


In [117]:
q='''SELECT * FROM  cleaned_nodetags WHERE id=461853706'''
sql_to_df(q)

,id,key,value,type
0,461853706,name,Walgreens,regular
1,461853706,amenity,pharmacy,regular


2.Name and number of Hospitals  by city in greater Houston area

In [119]:
q='''SELECT * FROM  cleaned_nodetags WHERE id IN (SELECT id FROM cleaned_nodetags WHERE value ='hospital') 
; '''
dfp1=sql_to_df(q)
q1='''SELECT * FROM  cleaned_nodetags WHERE id IN (SELECT id FROM cleaned_nodetags WHERE value ='hospital') 
AND key='city' ; '''
dfp2=sql_to_df(q1)
#dfp2
q2='''SELECT * FROM  cleaned_nodetags WHERE  id IN (SELECT id FROM cleaned_nodetags WHERE value ='hospital') 
AND key='name'; '''
dfp3=sql_to_df(q2)
dfp3
df4=pd.merge(dfp1,dfp2, on='id')
#df5=pd.merge(df4,df3,on='id')
df4.columns=['id','Key','Count','Type','Place',"City",'Type2']
#['value_y'].value_counts()
df5=pd.merge(df4,dfp3,on='id')

df6=df5[['Count','City','value']]
#df6.columns=['City','Name','Count']
df7=df6.groupby(['City','value']).count()
#df7=df7.rename(columns={'City':'City', 'value':'School Name', 'Amenity Type':'Count'})
df7

Count
City                value                                                   
Dickinson,Texas,USA Bay Area Recovery Center                              12
Houston,Texas,USA   Cullen Building                                        6
                    Doctors Hospital                                      11
                    Michael E. Debakey Center                              7
                    Park Plaza Hospital                                   14
                    St. John's Hospital (Houston Methodist)                7
                    Texas Children's Hospital                              7
                    Texas Heart Institute - Denton A. Cooley Building      9
Pasadena,Texas,USA  Patients St. Luke's Medical Center                     6
Tomball,Texas,USA   Texas Sports Medical Center                           10
                    Tomball Regional Hospital                             10
Webster,Texas,USA   Bay Area Regional Medical Center                       7

# Top 5 amenities  in Harris county

In [83]:
query='''SELECT a.id,a.key,a.value,b.value,count(a.value) as num
    FROM cleaned_nodetags as a, cleaned_nodetags as b
    WHERE a.id=b.id and a.key='amenity' and b.value='Harris'
    GROUP BY a.value ORDER BY num DESC LIMIT 5;'''
sql_to_df(query)

,id,key,value,value,num
0,368161440,amenity,fire_station,Harris,192
1,368162710,amenity,police,Harris,85
2,368162199,amenity,library,Harris,14
3,368162538,amenity,townhall,Harris,8
4,368160522,amenity,courthouse,Harris,6


1. Number and Name of Police stations in Harris County

In [85]:
query='''SELECT * FROM cleaned_nodetags WHERE value  LIKE '%Police%' AND key= 'name' AND id IN 
(SELECT id
    FROM cleaned_nodetags 
    WHERE  value='Harris'); '''
print "Number of police stations=",len(sql_to_df(query)) 
sql_to_df(query).head()


Number of police stations= 75


,id,key,value,type
0,368162236,name,Houston Police Department - Independence Heights,regular
1,368162237,name,Houston Police Department - Eastside Patrol,regular
2,368162240,name,Katy Police Department,regular
3,368162245,name,Houston Police Department - Southeast Patrol,regular
4,368162246,name,Galena Park Police Department,regular


2.Number and Name of fire stations in Harris County

In [84]:
query='''SELECT * FROM cleaned_nodetags WHERE value  LIKE '%Fire%' AND key= 'name' AND id IN 
(SELECT id
    FROM cleaned_nodetags 
    WHERE  value='Harris'); '''
print "Number of Fire stations=",len(sql_to_df(query)) 
sql_to_df(query).head()

Number of Fire stations= 193


,id,key,value,type
0,368160241,name,Pasadena Fire Chief,regular
1,368160461,name,Houston Fire Department Station 9,regular
2,368160463,name,Houston Fire Department Station 7,regular
3,368160464,name,Heights Fire Department,regular
4,368160572,name,Houston Fire Department - Arson Division,regular


3.Name of Airports in Harris County

In [35]:
query='''SELECT * FROM cleaned_nodetags WHERE value LIKE '%Airport' AND key= 'name' AND id IN 
(SELECT id
    FROM cleaned_nodetags 
    WHERE  value='Harris'); '''
sql_to_df(query)

,id,key,value,type
0,368162275,name,Houston Police Department - Hobby Airport,regular
1,368162708,name,Houston Police Department - Intercontinental A...,regular
2,369120474,name,Baytown Airport,regular
3,369120632,name,Ferris Airport,regular
4,369120828,name,Flying F Ranch Airport,regular
5,369120830,name,Roeder Airport,regular
6,369120831,name,Rogers Airport,regular
7,369121182,name,Green Acres Airport,regular
8,369121261,name,Skyhaven Airport,regular
9,369121619,name,West Houston Airport,regular


# Conclusion

After this investigation of the data set, I found that the open street map data for greater Houston area is incomplete. For example, Some amenities  (such as Walgreens pharmacy) which I personally know it does exist in Bellaire City, is not in the data. This might be due to incomplete representation in the data (i.e, the name Walgreens pharmacy may exist but its adress and city might not exist in the data). Another problem I found was strong non uniformity of the data. For example some amenities have full adresses including city, zip code even phone number and others have only names. This might be partly due to the data imported from the GNIS data base which is basically a database of names and also the fact that the open street map data is input of many contributers from GPS data. I suggest at least two ways to improve the quality of the data. One possible way is by importing cleaned and updated data from other sources such as google map APIs. Another way is to put an automated quality improvement system using machine learing techniques such as a recommender system that recommends attributes to a contributor and a feedback system that updates the changes every a new contributor made to other contributors for immediate review. I believe by putting those the above two methods with  a GPS data auditor,  a more robust data processor like data.py and my own auditing_city_and_street.py to input a more cleaner and uniform data into the open street map data. In general, implementing quality improvement in open street map data is beneficial  for getting a better map and better awareness of our world that would give countless advantages for the growing map demanding technologies. On the other hand, implementing quality improvement of OSM may be damaging if not properly and carefully handled. For example importing third-party datasets can't be beneficial if the data format is not well translated to the OSM data formats. Another damaging issue is problematic editing of the OSM data. In this case contributors may delete, change exiting features, tags  unintentionally that would result in poorer quality than it was. Therefore, if quality improvement needs tobe implemented it should be implemented with extra care and I suggest also to use autonomous recommending and feedback system in the OSM edition process.

# REFERENCES

1. https://github.com/allanbreyes/udacity-data-science/blob/master/p2/data.py
2. https://gist.github.com/carlward/54ec1c91b62a5f911c42#file-sample_project-md
3. http://wiki.openstreetmap.org/wiki/USGS_GNIS
4. http://flrec.ifas.ufl.edu/geomatics/wp-content/uploads/2012/02/tgis12037.pdf
5. https://pdfs.semanticscholar.org/f57e/61c9ec141196e9229bfd117518067e3d412a.pdf